In [1]:
import logging
import json
import re
import sys

    
def twomirror(direct):
        #  direction
    direction = str()

    # which side is reflective
    rightSide = bool()
    leftSide = bool()
    if not direct == "R" and not direct == "L":
        raise Exception("The " + direct + " direction of mirror is not supported.")
    direction = direct
    rightSide = True
    leftSide = True
    return direction, leftSide, rightSide
def onemirror(direct, side):
    #  direction
    direction = str()

    # which side is reflective
    rightSide = bool()
    leftSide = bool()
    direction = direct
    if not side == "R" and not side == "L":
        raise Exception("The " + side + " of a mirror is not supported.")
    if side == "R":
        leftSide = False
        rightSide = True
    if side == "L":
        rightSide = False
        leftSide = True
    return direction, leftSide, rightSide

In [2]:
#  used to track position in the path
class Position(object):
   
    col = int()
    row = int()
    orientation = str()

    #  "H" or "V"
    direction = str()

    #  "+": increase step; "-": decrease step
    def __init__(self, c, r, o, d):
      
        self.col = c
        self.row = r
        
        if not o == "H" and not o == "V":
            raise Exception("The " + o + " direction of mirror is not supported.")
        self.orientation = o
        if not d == "+" and not d == "-":
            raise Exception("The " + d + " direction of movement is not supported.")
        self.direction = d

    def __str__(self):
        
        print("inside position str")
       
        return "position: " + self.col + "x" + self.row + " (" + self.orientation + self.direction + ") "

    def equals(self, p):
      
        if self.col == p.col and self.row == p.row and self.orientation == p.orientation and self.direction == p.direction:
            return True
        else:
            return False

In [3]:
from utils import mirror
from utils import position
import array
import io
import sys


In [6]:

import array
import io
import logging
import sys
        
def mazePath(board, col, row, orientation):
        
        if int(col) < 0 or int(row) < 0 or int(col) >= len(board) or int(row) >= len(board[0]) or (not orientation == "H" and not orientation == "V"):
            print("incorrect input")
            return
        print ("The dimensions of board: " + str(len(board)) + " x " + str(len(board[0])))
        path = []
        direction = "+"
        path.append(position.Position(col, row, orientation, direction))
        print('Extry of laser', path[0])
        last = path[len(path) - 1]
        while (int(last.col) >= 0 and int(last.col) < len(board)) and (int(last.row) >= 0 and int(last.row) < len(board[0])):
            nextPosition(board, path)
            last = path[len(path) - 1]
        print("the path of the laser: ")

        i = 0
        while i < len(path) - 1:
            print(path[i])
            i += 1
        print('Final exit of laser', path[i-1])
            
def nextPosition(board, path):
        """ method nextPosition """
        prev = path[len(path) - 1]
        prevCol = prev.col
        prevRow = prev.row
        prevOrient = prev.orientation
        prevDirection = prev.direction
        nextCol = -1
        nextRow = -1
        nextOrient = prevOrient
        nextDirection = prevDirection

        if prevOrient == "H":
            nextCol = int(prevCol) + (1 if (prevDirection == "+") else - 1)
            nextRow = int(prevRow)

        if prevOrient == "V":
            nextRow = int(prevRow) + (1 if (prevDirection == "+") else - 1)
            nextCol = int(prevCol)

        if (nextCol >= 0 and nextCol < len(board)) and (nextRow >= 0 and nextRow < len(board[0])):
            mirror = board[nextCol][nextRow]
            if mirror != None:
                mirrordir = mirror.strip().split(":")[0]
                mirrorleft = mirror.strip().split(":")[1]
                mirrorright = mirror.strip().split(":")[2]
                if mirrordir == "R":
                    if mirrorright == 'True':
                        if prevOrient == "V" and prevDirection == "+":
                            nextOrient = "H"
                            nextDirection = "+"
                        if prevOrient == "H" and prevDirection == "-":
                            nextOrient = "V"
                            nextDirection = "-"
                    if mirrorleft == 'True':
                        if prevOrient == "V" and prevDirection == "-":
                            nextOrient = "H"
                            nextDirection = "+"
                        if prevOrient == "H" and prevDirection == "+":
                            nextOrient = "V"
                            nextDirection = "-"
                if mirrordir == "L":
                    if mirrorright == 'True':
                        if prevOrient == "V" and prevDirection == "-":
                            nextOrient = "H"
                            nextDirection = "+"
                        if prevOrient == "H" and prevDirection == "-":
                            nextOrient = "V"
                            nextDirection = "+"
                    if mirrorleft == 'True':
                        if prevOrient == "V" and prevDirection == "+":
                            nextOrient = "H"
                            nextDirection = "-"
                        if prevOrient == "H" and prevDirection == "+":
                            nextOrient = "V"
                            nextDirection = "-"
        next = position.Position(str(nextCol), str(nextRow), nextOrient, nextDirection)
        for p in path:
            if p == next:
                raise Exception("the laser is trapped in the maze.")
        
        path.append(next)
        

if __name__ == "__main__":
    

        #Logging configuration
        logging.basicConfig(filename='logs/debug.log',level=logging.DEBUG,format='%(asctime)s %(levelname)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')
        logging.disable(logging.DEBUG)
        
        mazeRow = 0
        mazeCol = 0
        mirrors = []
        laserStartRow = -1
        laserStartCol = -1
        laserOrientation = None
        inputCounter = 0

        # Adding the logic to read the file from runtime argument
        #if len(sys.args) > 1:
        #    file = sys.args[0]
        

        contents= open("input/Sample2.txt", 'r')
        logging.info('Opening the input file ')
        
        #  read the configuration for the mirror board and the laser
        #  TO-DO: to catch all kinds of input file exceptions.
        
        while True:
            line = contents.readline() 
            if not line:break 

            if "-1" in line:
                inputCounter += 1
                continue     
            if line == "\n":
                continue

        #  read maze size
            if inputCounter == 0:
                mazeSize = line.strip()
                mazeCol = mazeSize.strip().split(",")[0]
                mazeRow = mazeSize.strip().split(",")[1]

            #  read mirror
            if inputCounter == 1:

                newline = line.strip()
                mirrors.append(newline)

            #  read laser
            if inputCounter == 2:
                laser = line.strip()
                laserStartCol = (laser[0:1])
                laserStartRow = laser[2:3]
                laserOrientation = laser[3:4]

        contents.close()
        logging.info('closing the input file ' )

        mirrorMaze = [[None]*int(mazeCol) for i in range(int(mazeRow))]
        #mirrorMaze = [[]]
        for m in mirrors:
            if len(m) > 4:
              
                d = m[3:4]
                s = m[4:]
                direction, leftSide, rightSide = onemirror(d,s)
            else:
                d = m[3:4]
                direction, leftSide, rightSide = twomirror(d)
            col = m[0:1]
            row = m[2:3]  
            mirrorMaze[int(col)][int(row)] = direction + ":" + str(leftSide)+ ":" + str(rightSide)
        mazePath(mirrorMaze, laserStartCol, laserStartRow, laserOrientation)
        


The dimensions of board: 4 x 5
Extry of laser position: 0x2 (H+) 
the path of the laser: 
position: 0x2 (H+) 
position: 1x2 (H+) 
position: 2x2 (H+) 
position: 3x2 (V-) 
position: 3x1 (V-) 
position: 3x0 (V-) 
Final exit of laser position: 3x0 (V-) 


In [6]:

import requests
url = "http://localhost:8088/messages"
response = requests.get(url=url)
print(response.text)

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with LibreSSL 2.8.3. See: https://github.com/urllib3/urllib3/issues/2168

In [7]:
contents= open("input/Sample2.txt", 'r')
mazeRow = 0
mazeCol = 0
mirrors = []
laserStartRow = -1
laserStartCol = -1
laserOrientation = None
inputCounter = 0

while True:
    line = contents.readline() 
    if not line:break 
    
    if "-1" in line:
        inputCounter += 1
        continue     
    if line == "\n":
        continue
        
#  read maze size
    if inputCounter == 0:
        mazeSize = line.strip()
        mazeCol = mazeSize.strip().split(",")[0]
        mazeRow = mazeSize.strip().split(",")[1]

    #  read mirror
    if inputCounter == 1:

        newline = line.strip()
        mirrors.append(newline)
        
    #  read laser
    if inputCounter == 2:
        laser = line.strip()
        laserStartCol = (laser[0:1])
        laserStartRow = laser[2:3]
        laserOrientation = laser[3:4]
        
        

        

In [17]:
contents= open("input/Sample2.txt", 'r')
in_ = contents.read()
print(in_)

5,4

-1

1,2RR

3,2L

-1

0,2H

-1




In [18]:
with open("input/Sample.txt") as f:
    contents = f.read()
    print(contents)

5,4

-1

1,2RR

3,2L

-1

1,0V

-1


